## Beta Weighting a Portfolio

In [2]:
import datetime as dt
import pandas as pd
import numpy as np

from pandas_datareader import data as pdr
import matplotlib.pyplot as plt

import yfinance as yfin
yfin.pdr_override()


In [3]:
end = dt.datetime.now()
start =end - dt.timedelta(days=1000)


# Stock list
stockList = ['FLT', 'HLO', 'QAN', 'VAS', 'WEB']
stocks = [stock+'.AX' for stock in stockList] #Yahoo finance requires .AX for ASX
stocks = ['^AXJO'] + stocks



print(stocks)


['^AXJO', 'FLT.AX', 'HLO.AX', 'QAN.AX', 'VAS.AX', 'WEB.AX']


In [5]:
# Import data and compute log returns
df = pdr.get_data_yahoo(stocks, start, end)

lnReturns = np.log(df['Adj Close'] / df['Adj Close'].shift(1)).dropna()


[*********************100%%**********************]  6 of 6 completed


### Calculating Beta using Matrix Algebra: OLS

$$\beta = (X^{T}X)^{-1}X^{T}y$$

In our case, X is the vector of market index returns and y is our stock returns The matrix X is called the design matrix, and it is for our example a column vector. We're not regressing the market returns upon all the stocks together. Each stock has its own Beta!

However, when implemented, we add a column of 1's so the design matrix X rather has 2 columns to account for an intercept term.

Without the intercept, the fitted line is forced to cross the origin

In [8]:



def matrix_beta(df):
    # Market index is the first column 0
    X = df.values[:, [0]]
    
    # add an additional column for the intercept (initalise as 1's)
    X = np.concatenate([np.ones_like(X), X], axis=1)
    
    # Apply matrix algebra for linear regression model
    beta = np.linalg.pinv(X.T @ X) @ X.T @ df.values[:, 1:]
    
    return pd.Series(beta[1], df.columns[1:], name='Beta')

beta = matrix_beta(lnReturns)
beta

Ticker
HLO.AX    0.708975
QAN.AX    0.481246
VAS.AX    0.178948
WEB.AX    0.717300
^AXJO     0.178960
Name: Beta, dtype: float64

### Beta as a linear operator
Market-beta can be weighted, averaged, added, etc
$$\beta_{p,m}=w_{1}*\beta_{1}+w_{2}*\beta_{2}$$










### Including Options 
Beta relies on a linear model. An out of the money option may have a distinctly non-linear payoff. The change in price of an option relative to the change in the price of the underlying asset (for example a stock) is not constant.

To include options, we would require the Delta of the option

Then you could weight the Option Deltas by Beta
